In [ ]:
import os, cv2, sys
import random
import numpy as np
import pyautogui
import mss
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, StaleElementReferenceException, ElementNotInteractableException, ElementNotVisibleException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions
url="https://anketa.minzdrav.gov.ru/stacionar/2170c959-4e72-4d32-9a88-4d8ab66bfa0a/4397b2bc-46ca-4206-9f23-ba9716511f0d"

import yandexdriver_update
yandexdriver_update.yandexdriver_update()
tormoz = 5

def start_driver():
    binary_yandex_driver_file = 'yandexdriver.exe'

    profile_directory = os.path.join(os.getcwd(), "NOKUser")
    os.makedirs(profile_directory, exist_ok=True)
    
    options = webdriver.ChromeOptions()
    
    options.add_argument("force-device-scale-factor=0.5");
    options.add_argument("high-dpi-support=0.5");

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features=AutomationControlled")

    options.add_argument(f"--user-data-dir={profile_directory}")
    
    driver = webdriver.Chrome(options=options, service=Service(executable_path=binary_yandex_driver_file))
    
    driver.set_page_load_timeout(60)
    driver.get(url)

    time.sleep(10)
    print('current site:',driver.title)
    return driver

class ClickError(Exception):
    def __init__(self, xpath: str):
        self.xpath = xpath
        self.message = f"Failed to click element with xpath: {xpath}"
        super().__init__(self.message)
        

def locate_and_click_matching_images(reference_image_path, confidence=0.8, monitor_num=1, region=None, click_displacement_x=0.5, click_displacement_y=0.5,sleep_after=1):
    """
    Locate and click all instances of a reference image on a specific monitor within a region of interest.
    
    Parameters:
    - reference_image_path: Path to the reference image file
    - confidence: Matching threshold (0-1), higher means more strict matching
    - monitor_num: Monitor number to capture (1 for primary monitor)
    - region: Region of interest as (x, y, width, height) or None for full monitor
    
    Returns:
    - List of coordinates where matches were found and clicked
    """
    reference = cv2.imread(reference_image_path)
    if reference is None:
        print(f"Error: Could not load reference image from {reference_image_path}")
        return []

    reference_gray = cv2.cvtColor(reference, cv2.COLOR_BGR2GRAY)

    with mss.mss() as sct:
        monitor = sct.monitors[monitor_num]

        if region:
            x, y, width, height = region
            monitor = {
                "top": monitor["top"] + y,
                "left": monitor["left"] + x,
                "width": width,
                "height": height
            }
        
        screenshot = np.array(sct.grab(monitor))
        
    screenshot_gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2RGB)
    screenshot_gray = cv2.cvtColor(screenshot_gray, cv2.COLOR_RGB2GRAY)
    
    result = cv2.matchTemplate(screenshot_gray, reference_gray, cv2.TM_CCOEFF_NORMED)
    
    locations = np.where(result >= confidence)
    
    ref_height, ref_width = reference_gray.shape
    
    matches = []
    for pt in zip(*locations[::-1]):
        matches.append(pt)
    
    grouped_matches = []
    while matches:
        match = matches.pop(0)
        grouped_matches.append(match)
        
        non_overlapping = []
        for m in matches:
            if abs(m[0] - match[0]) > ref_width/2 or abs(m[1] - match[1]) > ref_height/2:
                non_overlapping.append(m)
        matches = non_overlapping
    
    clicked_positions = []
    for match in grouped_matches:
        center_x = match[0] + ref_width * click_displacement_x
        center_y = match[1] + ref_height * click_displacement_y
        
        screen_x = center_x + monitor["left"]
        screen_y = center_y + monitor["top"]
        
        pyautogui.moveTo(screen_x, screen_y, duration=0.2)
        pyautogui.click()
        
        clicked_positions.append((screen_x, screen_y))
        time.sleep(0.5) 
    print(f"Found and clicked {len(clicked_positions)} matching images")
    print(f"Clicked positions: {clicked_positions}")
    time.sleep(sleep_after) 
    if len(clicked_positions)>0:
        return True
    return False

def generate_level_ups(num):
    text=''
    for i in range(num):
        text+="/.."
    return text

def click_xpath(xpath,wait=10,crit=True):
    print("click_xpath", "crit", crit, xpath, end='')
    all_elements = driver.find_elements(By.XPATH, xpath)
    clickable_elements=[]
    for element in all_elements:
        try:
            if element.is_displayed() and element.is_enabled():
                is_in_viewport = driver.execute_script("""
                    var elem = arguments[0];
                    var rect = elem.getBoundingClientRect();
                    return (
                        rect.top >= 0 &&
                        rect.left >= 0 &&
                        rect.bottom <= (window.innerHeight || document.documentElement.clientHeight) &&
                        rect.right <= (window.innerWidth || document.documentElement.clientWidth)
                    );
                """, element)
                
                is_not_covered = driver.execute_script("""
                    var elem = arguments[0];
                    var rect = elem.getBoundingClientRect();
                    var x = rect.left + rect.width / 2;
                    var y = rect.top + rect.height / 2;
                    var elementAtPoint = document.elementFromPoint(x, y);
                    return elementAtPoint === elem || elem.contains(elementAtPoint);
                """, element)
                
                if is_in_viewport and is_not_covered:
                    clickable_elements.append(element)
        except (ElementNotVisibleException, ElementNotInteractableException):
            continue
        except Exception as e:
            print(f"Error checking element: {str(e)}")
            continue
    print(f' found {len(clickable_elements)} elements')
    if len(clickable_elements)<1:
        if crit:
            raise ClickError(xpath)
        return False
    for element in clickable_elements:
        try:
            element.click()
            return True
        except (ElementClickInterceptedException, StaleElementReferenceException):
            print("click_xpath intercepted", "crit", crit, xpath)
            time.sleep(1)
            if crit==True:
                counter=0
                
                while counter<3:
                    if click_xpath(xpath,wait=1,crit=False):
                        return True
                    counter+=1
                print("!!!!!!!!!!ClickError Exception!!!!!!!!!!", "crit", crit, xpath)
                raise ClickError(xpath)
            else:
                return False
        except (TimeoutException):
            print("!!!!!!!!!!click_xpath timeout!!!!!!!!!!", "crit", crit, xpath)
            if crit:
                raise ClickError(xpath)
            return False

def type_xpath(xpath,string,wait=10):
    print('typing', string, xpath)
    element = WebDriverWait(driver,wait).until(expected_conditions.element_to_be_clickable((By.XPATH, xpath)))
    time.sleep(0.3)
    element.clear()
    time.sleep(0.3)
    element.send_keys(string)
    time.sleep(1)
    return True

def click_text(text,level_ups=0, wait=10, crit=True):
    xpath = "//*[text()='"+ text+"']"+generate_level_ups(level_ups)
    return click_xpath(xpath, wait=wait, crit=crit)

def click_class(text, wait=10, crit=True, count=0):
    if count==0:
        number=""
    else:
        number="["+str(count)+"]"
    xpath = "(//*[contains(@class,'"+text+"')]"+number+")"
    return click_xpath(xpath, wait=wait, crit=crit)

def randomizator(probability):
    rnd=random.randrange(1,100)
    if rnd <= probability:
        print("yes")
        return 1
    else:
        print("---------------------------Noooooooooooooo!!!")
        return 0
def answer(yes):
    time.sleep(0.2)
    if yes:
        click_text('Да')
    else:
        click_text('Нет')
    time.sleep(tormoz)
    nxt()
    
def randomAnswer(probability):
    rnd=random.randrange(1,100)
    if rnd <= probability:
        print("yes")
        answer(1)
        return 1
    else:
        print("---------------------------Noooooooooooooo!!!")
        answer(0)
        return 0

def nxt():
    click_text(' Далее ')
    time.sleep(tormoz)

def send_anketa():
    print("1")#------------question 1-----------------
    if randomizator(1):
        print("---------------------------bingo!!!")
        click_text('плановая')
        nxt()#------------------------------------to question 1.1
        click_text('менее 7 календарных дней')
        nxt()#------------------------------------to question 1.2
        answer(1)#--------------------------------to question 1.3
        click_text('при обращении в медицинскую организацию')
        nxt()#------------------------------------to question 2 
        print("2")#------------question 2-----------------
        if not randomAnswer(95):
            badvariant=random.randrange(1,101)
            if badvariant<10:
                click_text('отсутствие свободных мест ожидания')
            elif badvariant<20:
                click_text('состояние гардероба')
            elif badvariant<30:
                click_text('состояние санитарно-гигиенических помещений')
            elif badvariant<40:
                click_text('отсутствие питьевой воды')
            else:
                click_text('санитарное состояние помещений')
            nxt()#-------------------------------to question 3
        print("3") #------------question 3-----------------
        randomAnswer(95)#---------------------to question 4
    else:
        click_text('экстренная')
        nxt()#------------------------------------to question 4
    print("4")#------------question 4---------------------
    if randomAnswer(3):
        print("4.1")#------------question 4.1-----------------
        print("---------------------------bingo!!!")
        invalid=random.randrange(1,100)
        if invalid < 20:
            click_text('I группа')
        elif invalid < 40:
            click_text('II группа')
        else:
            click_text('III группа')
        nxt()
        #------------question 4.2-----------------
        randomAnswer(95)
        #------------question 4.3-----------------
        randomAnswer(95)
    print("5")#------------question 5-----------------
    if randomAnswer(95):
        #------------question 5.1-----------------
        randomAnswer(95)
    print("6")#------------question 6-----------------
    if randomAnswer(95):
        #------------question 6.1-----------------
        randomAnswer(95)
    print("7")#------------question 7-----------------
    if not randomAnswer(95):
        #------------question 7.1-----------------
            badvariant=random.randrange(1,101)
            time.sleep(tormoz)
            if badvariant<30:
                click_text('питание')
            elif badvariant<40:
                click_text('отсутствие питьевой воды')
            elif badvariant<70:
                click_text('санитарное состояние помещений')
            elif badvariant<80:
                click_text('действия персонала по уходу')
            else:
                click_text('состояние санитарно-гигиенических помещений')
            nxt()
    print("8")#------------question 8-----------------
    randomAnswer(95)
    print("9")#------------question 9-----------------
    randomAnswer(95)
    print("10")#------------question 10-----------------
    randomAnswer(95)
    print("11")#------------question 11-----------------
    randomAnswer(95)
    print("12")#------------question 12-----------------
    if randomAnswer(95):
        #------------question 12.1-----------------
        randomAnswer(95)
    time.sleep(tormoz)
    click_text(' Отправить ')
    print("Clicking send")
    time.sleep(tormoz)
    click_text(' ОК ')
    print("Click OK")
    return True

def maybe_execute(func, probability=0.5, *args, **kwargs):
    """
    Execute a function with a given probability.
    
    Parameters:
    - func: The function to execute
    - probability: Float between 0 and 1 representing execution probability
    - *args, **kwargs: Arguments to pass to the function
    
    Returns:
    - Function result if executed, None otherwise
    """
    if random.random() < probability:
        return func(*args, **kwargs)
    return None

current dir: C:\Users\User\Anaconda\Github
No stored version found
Latest version: v25.10.0-stable
New version found: v25.10.0-stable
Searching for the latest YandexDriver Windows binary...
Found 30 releases. Searching for Windows binaries...

Found Windows binary in release v25.10.0-stable (published: 2025-10-31T19:14:57Z)
Binary: yandexdriver-25.10.0.2474-win64.zip (19.76 MB)


100%|██████████| 19.8M/19.8M [00:06<00:00, 3.39MB/s]

Successfully downloaded yandexdriver-25.10.0.2474-win64.zip to c:\Users\User\Anaconda\Github
Extracting yandexdriver-25.10.0.2474-win64.zip to current directory...
Found 1 files in archive
Extracted: yandexdriver.exe (1/1)
Successfully extracted 1 files to c:\Users\User\Anaconda\Github
Extracted files:
  - yandexdriver.exe


In [4]:
driver = start_driver()
monitor_number = 2
region_of_interest = (50, 50, 600, 600)  # (x, y, width, height) Leave as None to search the entire monitor
if not locate_and_click_matching_images(r"images\captcha.png", confidence=0.9, monitor_num=monitor_number, region=region_of_interest,click_displacement_x=0.1,click_displacement_y=0.7):
    sys.exit(1)
locate_and_click_matching_images(r"images\medic.png", confidence=0.85, monitor_num=monitor_number, region=region_of_interest)
locate_and_click_matching_images(r"images\ready.png", confidence=0.9, monitor_num=monitor_number, region=region_of_interest)
while True:
    if not locate_and_click_matching_images(r"images\captcha_error.png", confidence=0.9, monitor_num=monitor_number, region=region_of_interest):
        print('No error!')
        break
    print('Error!')
    locate_and_click_matching_images(r"images\medic.png", confidence=0.85, monitor_num=monitor_number, region=region_of_interest)
    locate_and_click_matching_images(r"images\ready.png", confidence=0.9, monitor_num=monitor_number, region=region_of_interest)
while True:
    send_anketa()
maybe_execute(send_anketa, 0.5)
maybe_execute(send_anketa, 0.5)
driver.quit()

SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
	GetHandleVerifier [0x0x7ff70f8412e5+24661]
	(No symbol) [0x0x7ff70f7827e0]
	(No symbol) [0x0x7ff70f5d15fa]
	(No symbol) [0x0x7ff70f60fe2d]
	(No symbol) [0x0x7ff70f60a44b]
	(No symbol) [0x0x7ff70f660cbf]
	(No symbol) [0x0x7ff70f660420]
	(No symbol) [0x0x7ff70f6539d3]
	(No symbol) [0x0x7ff70f61936f]
	(No symbol) [0x0x7ff70f61a713]
	GetHandleVerifier [0x0x7ff70fbf324d+3899325]
	GetHandleVerifier [0x0x7ff70fc20662+4084690]
	GetHandleVerifier [0x0x7ff70f917bf9+903529]
	GetHandleVerifier [0x0x7ff70f91f3ce+934206]
	(No symbol) [0x0x7ff70f78c2eb]
	(No symbol) [0x0x7ff70f787ea4]
	(No symbol) [0x0x7ff70f788057]
	(No symbol) [0x0x7ff70f778fa5]
	Bas